<a href="https://colab.research.google.com/github/werent4/deep_stock/blob/develop/LSTM_5_min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from keras.src.engine.training import optimizer
from tensorflow import keras
from tensorflow.keras import layers

# Dataset preperation

In [2]:
data = pd.read_csv('AAMC.csv')
data.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2023-08-15 09:30:00-04:00,10.7200,12.2600,10.6447,12.2600,12.2600,0
1,2023-08-15 09:35:00-04:00,12.7500,12.9800,11.8601,12.6500,12.6500,201711
2,2023-08-15 09:40:00-04:00,12.5682,12.5999,11.0801,11.9082,11.9082,32626
3,2023-08-15 09:45:00-04:00,11.9600,11.9600,11.3352,11.6900,11.6900,9042
4,2023-08-15 09:50:00-04:00,11.6800,12.9800,11.6800,12.7572,12.7572,40193


In [3]:
data.shape

(1193, 7)

In [4]:
data.isna().sum()

Datetime     0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [5]:
data['Datetime'] = data['Datetime'].str.replace('-04:00', '')
datetime = pd.to_datetime(data['Datetime'], format='%Y-%m-%d %H:%M:%S')

In [6]:
datetime.dt.year

0       2023
1       2023
2       2023
3       2023
4       2023
        ... 
1188    2023
1189    2023
1190    2023
1191    2023
1192    2023
Name: Datetime, Length: 1193, dtype: int64

In [7]:
data.sample(3)

,Datetime,Open,High,Low,Close,Adj Close,Volume
302,2023-08-22 11:10:00,8.70,8.7000,8.59,8.6800,8.6800,1578
182,2023-08-17 15:35:00,10.11,10.1100,10.08,10.0800,10.0800,578
769,2023-09-14 10:00:00,7.00,7.1045,7.00,7.0886,7.0886,3861


In [8]:
data['Target_price'] = data['Close'].shift(-1) # create target feature
data = data.iloc[:-1]# drop last elt (для него нету данных)

In [9]:
target_price = data['Target_price'].values
features = data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]#.values

In [10]:
features['Year'] = datetime.dt.year
features['Month'] = datetime.dt.month
features['Day'] = datetime.dt.day
features['Hour'] = datetime.dt.hour
features['Minute'] = datetime.dt.minute

<ipython-input-10-29af54073ed1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Year'] = datetime.dt.year


In [11]:
features.sample(3)

,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day,Hour,Minute
1173,6.635,6.6611,6.60,6.6611,6.6611,3318,2023,10,12,13,15
167,10.070,10.0700,10.07,10.0700,10.0700,206,2023,8,17,13,10
461,9.240,9.2700,9.24,9.2600,9.2600,1556,2023,8,28,15,0


In [12]:
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

In [13]:
# Define the number of time steps and features # 0.0247
n_time_steps = round(len(data) / 2)
n_features = features.shape[1]

In [14]:
X, y = [], []
for i in range(n_time_steps, len(data)):
    X.append(features[i - n_time_steps:i])
    y.append(target_price[i])

In [15]:
X = np.array(X)
y = np.array(y)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
X_train.shape

(476, 596, 11)

In [18]:
y_train.shape

(476,)

# Model creation

In [19]:
model = keras.models.Sequential()
model.add(layers.LSTM(50, activation='tanh', input_shape=(n_time_steps, n_features)))
model.add(layers.Dense(1))


In [20]:
loss = 'mean_squared_error' #keras.losses.SparseCategoricalCrossentropy(from_logits= True)
optim = keras.optimizers.Adam(learning_rate= 0.001)
metrics = [""]

In [21]:
model.compile(optimizer= optim, loss= loss, )

In [22]:
model.fit(X_train, y_train, epochs=50, batch_size= 32)

Epoch 1/50
15/15 [==============================] - 6s 242ms/step - loss: 38.9735
Epoch 2/50
15/15 [==============================] - 4s 246ms/step - loss: 11.7452
Epoch 3/50
15/15 [==============================] - 3s 228ms/step - loss: 1.6211
Epoch 4/50
15/15 [==============================] - 4s 224ms/step - loss: 0.1087
Epoch 5/50
15/15 [==============================] - 4s 300ms/step - loss: 0.1395
Epoch 6/50
15/15 [==============================] - 3s 202ms/step - loss: 0.0866
Epoch 7/50
15/15 [==============================] - 3s 202ms/step - loss: 0.0767
Epoch 8/50
15/15 [==============================] - 3s 200ms/step - loss: 0.0762
Epoch 9/50
15/15 [==============================] - 4s 298ms/step - loss: 0.0752
Epoch 10/50
15/15 [==============================] - 3s 201ms/step - loss: 0.0749
Epoch 11/50
15/15 [==============================] - 3s 201ms/step - loss: 0.0748
Epoch 12/50
15/15 [==============================] - 5s 329ms/step - loss: 0.0746
Epoch 13/50
15/15 [====

In [23]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
f'Test Loss: {loss}'


4/4 [==============================] - 1s 64ms/step - loss: 0.0814


'Test Loss: 0.08135940879583359'